In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

quantization_config = BitsAndBytesConfig(load_in_8bit = True)

tokenizer = AutoTokenizer.from_pretrained("TheBloke/meditron-7B-GPTQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/meditron-7B-GPTQ", device_map='auto')

# tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/nlp/local_meditron_model")
# model = AutoModelForCausalLM.from_pretrained('/home/ubuntu/nlp/local_meditron_model')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using cuda device


/home/ubuntu/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/ubuntu/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/ubuntu/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/home/ubuntu/miniforge3/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/meditron-7B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.laye

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [2]:
tokenizer, model 

(LlamaTokenizerFast(name_or_path='TheBloke/meditron-7B-GPTQ', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<PAD>', 'cls_token': '<CLS>', 'mask_token': '<MASK>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	32000: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32001: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32002: AddedToken("<EOD>", rstrip=False, lstrip=Fa

In [3]:
model.save_pretrained("./local_meditron_model")
tokenizer.save_pretrained("./local_meditron_model")

('./local_meditron_model/tokenizer_config.json',
 './local_meditron_model/special_tokens_map.json',
 './local_meditron_model/tokenizer.model',
 './local_meditron_model/added_tokens.json',
 './local_meditron_model/tokenizer.json')

In [4]:
import torch
import gc
from datasets import load_from_disk

def load_dataset_and_create_base_prompt(dataset_name = "MedQA-USMLE-4-options"):
    dataset = load_from_disk(dataset_name)
    num_shots = 3     # Number of few-shot examples to use


    # Get few-shot examples from the training set
    few_shot_examples = dataset['train'].select(range(num_shots))

    # remove the few-shot examples from the dataset
    dataset['train'] = dataset['train'].select(range(num_shots, len(dataset['train'])))

    # Create the prompt with few-shot examples
    prompt = "The following are multiple choice questions (with answers) about medical:\n\n"
    for ex in few_shot_examples:
        prompt += f"Question: {ex['question']}\nChoices:\n"
        for opt, value in ex['options'].items():
            prompt += f"{opt}. {value}\n"
        prompt += f"Correct Answer:{ex['answer_idx']}\n\n"

    return dataset, prompt

raw_dataset, base_prompt = load_dataset_and_create_base_prompt()
print(base_prompt), print(raw_dataset)

The following are multiple choice questions (with answers) about medical:

Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
Choices:
A. Ampicillin
B. Ceftriaxone
C. Doxycycline
D. Nitrofurantoin
Correct Answer:D

Question: A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have pr

(None, None)

In [5]:
def create_prompt_and_return_correct_answer(item, prompt=base_prompt):
    # add to the prompt the first question from the dataset
    prompt += f"Question: {item['question']}\nChoices:\n"
    for opt, value in item['options'].items():
        prompt += f"{opt}. {value}\n"
    correct_answer_letter = item['answer_idx']
    # empty answer
    prompt += f"Correct Answer:"    
    # print(f"finale prompt to be sent to model is: {prompt}")
    # tokenize the prompt
    return prompt, correct_answer_letter

example_prompt, example_answer = create_prompt_and_return_correct_answer(raw_dataset['train'][0], base_prompt)
print(example_prompt), print(example_answer), print(raw_dataset['train'][0])

The following are multiple choice questions (with answers) about medical:

Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
Choices:
A. Ampicillin
B. Ceftriaxone
C. Doxycycline
D. Nitrofurantoin
Correct Answer:D

Question: A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have pr

(None, None, None)

In [6]:
import logging
logging.basicConfig(level=logging.DEBUG)

def get_probabilities(prompt):
    with torch.no_grad():
        
        # encode the prompt
        prompt = tokenizer(prompt, return_tensors="pt").to(device)
        
        # generate the answer to get the logits
        logits = model(**prompt).logits

        # only care about the last projection in the last batch
        logits = logits[-1, -1].clone().detach().cpu()


        # softmax() to get probabilities
        probs = torch.nn.functional.softmax(logits, dim=-1)
        
        # dict of probabilities with token names
        probs, ids = torch.sort(probs, descending=True)

        # keep only the top k
        k = 10
        probs_top_k = probs[:k]
        ids_top_k = ids[:k]
        texts_top_k = tokenizer.convert_ids_to_tokens(ids_top_k)

        # print
        logging.debug(f"top {k} tokens with probabilities:")
        for prob, text in zip(probs_top_k, texts_top_k):
            logging.debug(f"{prob:.4f}: \"{text}\"")
        
        # make the key value in probs
        probs_name_list = [(probs[i].item(), tokenizer.convert_ids_to_tokens(ids[i].item())) for i in range(len(probs))]
        
        # make probs_name_value a list of tuples to be dictionary
        probs_name_dict = dict(probs_name_list)
        
        # switch the key and value
        name_probs_dict = {v: k for k, v in probs_name_dict.items()}
                
        # if A, B, C or D not exist set probability to 0
        for letter in ['A', 'B', 'C', 'D']:
            if letter not in name_probs_dict:
                name_probs_dict[letter] = 0.0

        return {"A": name_probs_dict['A'], "B": name_probs_dict['B'], "C": name_probs_dict['C'], "D": name_probs_dict['D']}


example_probabilities = get_probabilities(example_prompt)

# get the answer with the highest probability
answer_with_max_prob = max(example_probabilities, key=example_probabilities.get)

# get the correct answer probability
correct_answer_prob = example_probabilities[example_answer]

print(example_probabilities), print(answer_with_max_prob), print(example_answer), print(correct_answer_prob), print(raw_dataset['train'][0])

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
DEBUG:root:top 10 tokens with probabilities:
DEBUG:root:0.3108: "C"
DEBUG:root:0.2383: "B"
DEBUG:root:0.2238: "D"
DEBUG:root:0.2038: "A"
DEBUG:root:0.0046: "▁A"
DEBUG:root:0.0039: "▁C"
DEBUG:root:0.0039: "▁B"
DEBUG:root:0.0038: "▁D"
DEBUG:root:0.0031: "E"
DEBUG:root:0.0030: "<0x0A>"


{'A': 0.20379683375358582, 'B': 0.23826263844966888, 'C': 0.3107530474662781, 'D': 0.2238270342350006}
C
A
0.20379683375358582
{'question': 'A pulmonary autopsy specimen from a 58-year-old woman who died of acute hypoxic respiratory failure was examined. She had recently undergone surgery for a fractured femur 3 months ago. Initial hospital course was uncomplicated, and she was discharged to a rehab facility in good health. Shortly after discharge home from rehab, she developed sudden shortness of breath and had cardiac arrest. Resuscitation was unsuccessful. On histological examination of lung tissue, fibrous connective tissue around the lumen of the pulmonary artery is observed. Which of the following is the most likely pathogenesis for the present findings?', 'answer': 'Thromboembolism', 'options': {'A': 'Thromboembolism', 'B': 'Pulmonary ischemia', 'C': 'Pulmonary hypertension', 'D': 'Pulmonary passive congestion'}, 'meta_info': 'step1', 'answer_idx': 'A', 'metamap_phrases': ['pulm

(None, None, None, None, None)

In [8]:
# create pandas dataframe to store the results
import pandas as pd
import tqdm
df = pd.DataFrame(columns=['prompt', 'correct_answer', 'answer_with_max_prob', 'prob_of_correct_answer', 'probabilities'])

# iterate over the training set and get the prompt, the correct answer and the probabilities
logging.getLogger().setLevel(logging.INFO)
i = 0
for item in tqdm.tqdm(raw_dataset['train']):
    # i += 1
    # if i < 7663:
    #     continue
    # if i == 7663:
    #     print("starting from 7660")
    prompt, correct_answer = create_prompt_and_return_correct_answer(item, base_prompt)
    probabilities = get_probabilities(prompt)
    answer_with_max_prob = max(probabilities, key=probabilities.get)
    prob_of_correct_answer = probabilities[correct_answer]
    logging.debug(f"prompt {prompt}\nprobabilities {probabilities}\ncorrect_answer {correct_answer}\nanswer_with_max_prob {answer_with_max_prob}\n\n")
    df = df._append({'prompt': prompt, 'correct_answer': correct_answer, 'answer_with_max_prob': answer_with_max_prob, 'prob_of_correct_answer': prob_of_correct_answer, 'probabilities': probabilities}, ignore_index=True)
print(df.head())
 # save the dataframe
df.to_csv('meditron.csv', index=False)

  0%|          | 0/10175 [00:00<?, ?it/s]/tmp/ipykernel_3335/3072891525.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append({'prompt': prompt, 'correct_answer': correct_answer, 'answer_with_max_prob': answer_with_max_prob, 'prob_of_correct_answer': prob_of_correct_answer, 'probabilities': probabilities}, ignore_index=True)
100%|██████████| 10175/10175 [4:34:46<00:00,  1.62s/it] 


                                              prompt correct_answer  \
0  The following are multiple choice questions (w...              A   
1  The following are multiple choice questions (w...              D   
2  The following are multiple choice questions (w...              C   
3  The following are multiple choice questions (w...              D   
4  The following are multiple choice questions (w...              A   

  answer_with_max_prob  prob_of_correct_answer  \
0                    C                0.203797   
1                    C                0.280022   
2                    C                0.340576   
3                    C                0.288692   
4                    C                0.217726   

                                       probabilities  
0  {'A': 0.20379683375358582, 'B': 0.238262638449...  
1  {'A': 0.14080394804477692, 'B': 0.211371764540...  
2  {'A': 0.17668865621089935, 'B': 0.168597489595...  
3  {'A': 0.21121151745319366, 'B': 0.126120164990...